Harvey Flood Discrepancy Analysis
by Ava Erickson, Olivia G., Noah P., and Julissa R.

In [1]:
import arcpy
from arcpy import env
#import spatial analyst package so that we can process rasters
from arcpy import sa
#import everything from spatial analyst
from arcpy.sa import *
#import operating system
import os
#allows us to overwrite so we can run the code multiple times and freshly save our output each time
env.overwriteOutput = True

In [2]:
#make sure spatial analyst is working
arcpy.CheckOutExtension("Spatial")

'CheckedOut'

In [3]:
path = r'C:\Users\user\Box Sync\Geoprocessing\FinalProject'

In [4]:
#set the workspace environment to our project path
env.workspace = path

In [5]:
#create a list of tif files from our project folder
rastList = [f for f in os.listdir(r'C:\Users\user\Box Sync\Geoprocessing\FinalProject') if f.endswith('.tif')]
rastList

['Extract_harvey_2.tif',
 'flood_accuracy.tif',
 'flood_actual.tif',
 'flood_actual_utm.tif',
 'flood_discrepancy.tif',
 'flood_discrepency.tif',
 'flood_discrepency1.tif',
 'flood_model.tif',
 'flood_model_01.tif',
 'flood_model_clip.tif',
 'harvey_actual.tif',
 'HWE_FtBend.tif',
 'HWE_FtBend_Mask.tif',
 'HWE_reclass.tif',
 'HW_3.tif',
 'intersected.tif',
 'minus.tif',
 'minus1.tif',
 'minus2.tif',
 'minus3.tif',
 'minus_inverted.tif']

In [6]:
#create a list of all shapefiles from our project folder
shpList = [f for f in os.listdir(r'C:\Users\user\Box Sync\Geoprocessing\FinalProject') if f.endswith('.shp')]
shpList

['census_clip.shp',
 'County.shp',
 'FloodplainsExport.shp',
 'ftb_blocks.shp',
 'ftb_tracts.shp',
 'ft_bend_tracts.shp',
 'ft_bend_utm.shp',
 'PolygonDiscrepancy.shp',
 'tracts.shp',
 'TractsWithColumn.shp',
 'tract_ftb.shp',
 'Tx_Census_BlockGroupsJurisdictional_TIGER.shp']

In [7]:
#create a variable for the Fort Bend county boundary, which has coordinate system UTM 14N
fort_bend = r'C:\Users\user\Box Sync\Geoprocessing\FinalProject' +'\\' + shpList[6]
fort_bend

'C:\\Users\\user\\Box Sync\\Geoprocessing\\FinalProject\\ft_bend_utm.shp'

In [8]:
#in ArcPro we used the raster calculator to write a con statement ensuring that both harvey_actual and flood_model
#have data = 1 and specific no data values
#for harvey_actual IsNull = 255
#for flood_model IsNull = 10

In [9]:
#create a variable for the harvey actual raster, which has values for both data and no data
#with Fort Bend county boundary as the extent
harvey_actual = r'C:\Users\user\Box Sync\Geoprocessing\FinalProject' +'\\' + rastList[10]
harvey_actual

'C:\\Users\\user\\Box Sync\\Geoprocessing\\FinalProject\\harvey_actual.tif'

In [10]:
#create a variable for the flood model raster, which has values for both data and no data
#with Fort Bend county boundary as the extent
flood_model = r'C:\Users\user\Box Sync\Geoprocessing\FinalProject' + '\\' + rastList[8]
flood_model

'C:\\Users\\user\\Box Sync\\Geoprocessing\\FinalProject\\flood_model_01.tif'

In [11]:
#create a conditional statement in which cells are selected when the flood model raster displays no data AND
#the harvey actual displays data = 1
output_raster = Con((Raster(flood_model)== 10) & (Raster(harvey_actual) == 1), 1, 0)

#save the output raster as flood_discrepancy
output_raster.save("flood_discrepancy.tif")

#the resulting raster displays the cells in which the flood model failed to predict flooding
#that occured during Hurricane Harvey

In [12]:
#create a variable for the discrepnacy raster
discrepancy = r'C:\Users\user\Box Sync\Geoprocessing\FinalProject' +'\\' + rastList[4]
discrepancy

'C:\\Users\\user\\Box Sync\\Geoprocessing\\FinalProject\\flood_discrepancy.tif'

In [37]:
#Create a loop that calculates the percent of each tract that was unexpectedly flooded

#Create variables for:
#Fort Bend tracts shapefile
tracts = r"C:\Users\user\Box Sync\Geoprocessing\FinalProject\ftb_tracts.shp"
#Discrepancy area shapefile, which was created in ArcPro using raster to polygon tool
discrepancy_shp = r"C:\Users\user\Box Sync\Geoprocessing\FinalProject\cellthatworks\PolygonDiscrepancy.shp"

#Initialize an empty list, which will hold the resulting percentages
results = []

#Create a new field in the tracts attribute table which will store the percentage results: 
#Name the new field
field_name = 'FloodPct' 
#Using list comprehensin, make sure that the new field (field_name) does not already exist in tracts, 
#this prevents errors when we run the code multple times
if field_name not in [f.name for f in arcpy.ListFields(tracts)]:
    #Use the function AddField to add the new field (field_name) 
    #to tracts, which will have datatype DOUBLE
    arcpy.AddField_management(tracts, field_name, 'DOUBLE')


#Use a search cursor to look through the tracts attribute table:     
#we are looking for the FID to be an identifier for each item and using the token SHAPE@ which gets an object 
#that contains the shapes geometry (this is useful because the object has an area)
with arcpy.da.SearchCursor(tracts, ['FID', 'SHAPE@']) as tract_cursor:
    #as we iterate through the tract attribute table, we will retrieve a tuple of FID and SHAPE@ for each item
    for tract_row in tract_cursor:
        #create the variable tract_fid to store FID value
        tract_fid = tract_row[0]
        #create the variable tract_geometry to store the SHAPE@ (shape geometry object)
        tract_geometry = tract_row[1]
        
        #Create a temporary (in memory) output feature class to store each clipped discrepancy area (clipped by tract):
        #use .format function to make the output name dynamic for each discreancy area that is clipped to a tract, 
        #the {} will be replaced with that tract's FID
        temp_discrep_clip = r'in_memory\flood_clip_{}'.format(tract_fid)
        
        #Clip the discrepancy area (discrepancy_shp) by each tract's geometry object (SHAPE@) using .clip function 
        #and save the output as temp_discrep_clip (unique path for outputs)
        arcpy.analysis.Clip(discrepancy_shp, tract_geometry, temp_discrep_clip)
        
        #First, see if the tract clip has any discrepancy area within it:
        #Use int to ensure the .GetCount result is an integer (so we can use >0)
        #Use the .GetCount function to return the number of features in the current discrepancy clip
        #use index 0 to retrieve the results from .GetCount (total number of rows in the current discrep clip table)
        #If the number of rows (features) is >0, continue 
        if int(arcpy.management.GetCount(temp_discrep_clip)[0]) > 0:
            #Use the .area function to find the area of current tract_geometry, 
            #save as the variable tract_area
            tract_area = tract_geometry.area
            #Initialize the variable discrep_area, which will accumulate the areas of all the discrepancy features 
            #within the current clip
            discrep_area = 0
            #Use a search cursor to look through the discrepancy clip attribute table 
            #and retrieve a geometry object (SHAPE@)
            with arcpy.da.SearchCursor(temp_discrep_clip, ['SHAPE@']) as discrep_cursor:
                #move through the table
                for discrep_row in discrep_cursor:
                    #add all of the discrapncy feature areas (using .area function) 
                    #to the discrep_area variable (to get a total discrepancy area)
                    discrep_area += discrep_row[0].area
            
            #Calculate the percentage of discrepancy within the current tract and 
            #save it as discrep_percentage
            discrep_percentage = (discrep_area / tract_area) * 100
        #If the integer result of .GetCount is not >0, then there was no discrepancy area in that tract and % is 0
        else:
            discrep_percentage = 0 
        
        #Use the .append function to add the current tract FID and associated 
        #discrepancy percentage to our results list as a tuple
        results.append((tract_fid, discrep_percentage))
        
        #Delete the temporary discrepancy clip so we can move on to the next tract using .delete function
        arcpy.management.Delete(temp_discrep_clip)

#Use an update cursor to add the discrepancy % to the tracts attribute table, 
#looking at FID and the new field_name 'FloodPct'
with arcpy.da.UpdateCursor(tracts, ['FID', field_name]) as update_cursor:
    #move through the tracts table
    for update_row in update_cursor:
        #create the variable tract_fid to store the FID variable
        tract_fid = update_row[0]
        #Match the discrep percentage result using list comprehension:
        #In our results list (defined above), return f[1] (discrep_percentage) if f[0] (tract_fid) equals the current tract_fid
        #This ensures that the correct percentage is matched with the current FID
        #Save the resulting % as discrep_percentage
        discrep_percentage = [f[1] for f in results if f[0] == tract_fid][0]
        #Update the new field (FLoodPct) as the discrep_percentage value for the in memory update_row
        update_row[1] = discrep_percentage
        #Use the .updateRow function to write the update_row (a tuple where [0] = tract_fid and [1] = discrep_percentage) 
        #into the tract attribute table as the update cursor
        update_cursor.updateRow(update_row)

#Use a loop to print all of the results from the results list
for tract_fid, discrep_percentage in results:
    print(f'{discrep_percentage:.2f}% of Tract {tract_fid} was unexpectedly flooded.')

12.09% of Tract 0 was unexpectedly flooded.
3.32% of Tract 1 was unexpectedly flooded.
0.00% of Tract 2 was unexpectedly flooded.
10.56% of Tract 3 was unexpectedly flooded.
11.34% of Tract 4 was unexpectedly flooded.
11.58% of Tract 5 was unexpectedly flooded.
4.51% of Tract 6 was unexpectedly flooded.
20.28% of Tract 7 was unexpectedly flooded.
0.00% of Tract 8 was unexpectedly flooded.
0.00% of Tract 9 was unexpectedly flooded.
0.00% of Tract 10 was unexpectedly flooded.
12.16% of Tract 11 was unexpectedly flooded.
0.00% of Tract 12 was unexpectedly flooded.
3.64% of Tract 13 was unexpectedly flooded.
26.67% of Tract 14 was unexpectedly flooded.
9.94% of Tract 15 was unexpectedly flooded.
1.65% of Tract 16 was unexpectedly flooded.
0.00% of Tract 17 was unexpectedly flooded.
25.57% of Tract 18 was unexpectedly flooded.
0.00% of Tract 19 was unexpectedly flooded.
26.20% of Tract 20 was unexpectedly flooded.
28.13% of Tract 21 was unexpectedly flooded.
1.04% of Tract 22 was unexpected